In [ ]:
import bdat
import pandas as pd
import altair as alt

In [ ]:
## import data

# build metadata
dataspec = bdat.DataSpec(
    "neware",
    durationColumn=bdat.TimeColumnSpec("Time##T_+01_00", bdat.Timestamp()),
    currentColumn=bdat.ColumnSpec("Current#A#D"),
    voltageColumn=bdat.ColumnSpec("Voltage#V#D"),
    chargeSpec=bdat.SeparateColumns(bdat.ColumnSpec("Charge Capacity#Ah#D"), bdat.ColumnSpec("Discharge Capacity#Ah#D")),
)
batterytype = bdat.BatterySpecies("Samsung 35E", capacity=3.5, endOfChargeVoltage=4.2, endOfDischargeVoltage=2.7)
battery = bdat.Battery("Samsung_35E_0001", None, batterytype, None, None, None)
cycling = bdat.Cycling("cycling example", None, None, None, battery, None, None, None, None, None, None)

# load cycling data from file
data = bdat.CyclingData(cycling, pd.read_parquet("1482756.parquet"), dataspec, 0, 0, 0)

In [ ]:
## segment cycling data into steps

# find steps
steplist = bdat.steps.find_linear_steps(data)

# plot steps
stepplot = bdat.plots.plot_steps(None, steplist, data.df)
alt.Chart.from_dict(stepplot.plot)

In [ ]:
## find patterns in the test steps

# specify patterns to look for
patterntypes = [
    bdat.patterns.Captest(),
    bdat.patterns.Testinfo(),
    bdat.patterns.UniformCycling(),
]

# search the test steps for matches
testeval = bdat.patterns.find_patterns(steplist, patterntypes, data)

# plot results
patternplot = bdat.plots.plot_testevals(None, testeval, data.df)
alt.Chart.from_dict(patternplot.plot)

In [ ]:
for eval in testeval.evals:
    if isinstance(eval, bdat.entities.patterns.UniformCyclingEval):
        print(eval.cyclecount)
